In [ ]:
from pulser import Pulse, Sequence, Register
from pulser.devices import MockDevice, AnalogDevice
import numpy as np
import matplotlib.pyplot as plt
from pulser.waveforms import InterpolatedWaveform
import pulser
import pulser_pasqal
def run_experiment(backend):
    result = backend.run()
    counts = result.sample_final_state(1000)
    return counts

C6 = 1e9  # Hz·μm^6
d0 = 4.0  # μm
h = d0 * np.sqrt((1/2)*(1/3) - 1/4)
V0 = C6 / d0**6

# Atom coordinates
coords = [(-d0/2, 0), (d0/2, 0), (0, h)]
reg = Register.from_coordinates(coords, prefix="q")

# Create the sequence with updated device
seq = Sequence(reg, MockDevice)
seq.declare_channel("rydberg", "rydberg_global")

# Choose NAND output: set to 1 (default NAND behavior) or 0 (flipped output)
target_output = 1

# Detuning ramp setup
duration = 2000  # ns
timesteps = 1000
t = np.linspace(0, duration, timesteps)

# Detunings (adiabatic ramps)
delta_1 = np.linspace(2 * V0, 0, timesteps)
delta_2 = np.linspace(2 * V0, 0, timesteps)
delta_3 = np.linspace(3 * V0 * (1 if target_output else 1), timesteps)
print(delta_3)
from pulser.waveforms import InterpolatedWaveform
from pulser.pulse import Pulse

# Use the output qubit’s detuning waveform (others are omitted for simplicity)
detuning_waveform = InterpolatedWaveform(duration, delta_3)
pulse = Pulse.ConstantDetuning(detuning_waveform,3*V0, phase=0)

seq.add(pulse, "rydberg")
seq.draw()

counts = run_experiment(pulser.backends.QutipBackend(seq))

# Let's plot the histogram associated to the measurements
# Let's select only the states that are measured more than 10 times
print(counts)
most_freq = {k: v for k, v in counts.items() if v > 10}
plt.bar(list(most_freq.keys()), list(most_freq.values()))
plt.xticks(rotation="vertical")
plt.show()

c:\Users\Kuift\AppData\Local\Programs\Python\Python312\Lib\site-packages\pulser\register\register.py:58: DeprecationWarning: Usage of `int`s or any non-`str`types as `QubitId`s will be deprecated. Define your `QubitId`s as `str`s, prefer setting `prefix='q'` when using classmethods, as that will become the new default once `int` qubit IDs become invalid.
  super().__init__(qubits, **kwargs)


ValueError: The pulse's amplitude goes over the maximum value allowed for the chosen channel.